# Members Data  
**MemberID:**Member pseudonym.<br>
**AgeAtFirstClaim:**Age in years at the time of the first claim’s date of service computed from  the date of birth; Generalized into ten year age intervals. <br>
**Sex:**Biological sex of member: M = Male; F=Female.

# Claims (Level 2) Data
**MemberI:**Member pseudonym.<br>
**ProviderID:**Provider pseudonym.<br>
**Vendor:**Vendor pseudonym.<br>
**PCP:**Primary care physician pseudonym.<br>
**Year:**Year in which the claim was made: Y1; Y2; Y3.<br>
**Specialty:**Generalized specialty.<br>
**PlaceSvc:**Generalized place of service.<br>
**PayDelay:**Number of days delay between the date of service (the date the actual  procedure was performed or service provided) and date of payment. Values  above 161 days (the 95% percentile) are top-coded as “162+”.<br>
**LengthOfStay:**Length of stay (discharge date – admission date + 1), generalized to: days up  to six days; (1-2] weeks; (2-4] weeks; (4-8] weeks; (8-12 weeks]; (12-26]  weeks; more than 26 weeks (26+ weeks). <br>
**DSFS:**Days since first claim, computed from the first claim for that member for  each year, generalized to: [0-1] month, (1-2] months, (2-3] months, (3-4]  months, (4-5] months, (5-6] months, (6-7] months, (7-8] months, (8-9]  months, (9-10] months, (10-11] months, (11-12] months.<br>
**PrimaryConditionGroup:**Broad diagnostic categories, based on the relative similarity of diseases and  mortality rates, that generalize the primary diagnosis codes (ICD-9-CM) [2]. <br>
**CharlsonIndex:**A measure of the affect diseases have on overall illness, grouped by  significance, that generalizes additional diagnoses. Scores greater than zero  are carried forward (for up to a year) in subsequent claims with a co morbidity score of zero [1,4]. <br>
**ProcedureGroup:**Broad categories of procedures, grouped according to the hierarchical  structure defined by the Current Procedural Terminology (CPT) [3]. <br>
**SupLOS:** Indicates if the NULL value for the LengthOfStay variable is due to  suppression done during the de-identification process. A value of 1 indicates  that suppression was applied. <br>

# Drug Count Data  
**MemberID:**Member pseudonym.<br>
**Year:**Year in which the drug prescription was filled: Y1; Y2; Y3.
**DSFS:** Days since first service (or claim), computed from the first claim for that  member for each year, generalized to: [0-1] month, (1-2] months, (2-3]  months, (3-4] months, (4-5] months, (5-6] months, (6-7] months, (7-8]  months, (8-9] months, (9-10] months, (10-11] months, (11-12] months. <br>
**DrugCount:** Count of unique prescription drugs filled by DSFS. No count is provided if  prescriptions were filled before DSFS zero. Values above 6, the 95%  percentile after excluding counts of zero, are top-coded as “7+”.<br>

#Lab Count Data  
**MemberID:**Member pseudonym.<br>
**Year:** Year in which the drug prescription was filled: Y1; Y2; Y3.<br>
**DSFS:** Days since first service (or claim), computed from the first claim for that  member for each year, generalized to: [0-1] month, (1-2] months, (2-3]  months, (3-4] months, (4-5] months, (5-6] months, (6-7] months, (7-8]  months, (8-9] months, (9-10] months, (10-11] months, (11-12] months.<br>
**LabCount:**Count of unique laboratory and pathology tests by DSFS. Values above 9,  the 95% percentile after excluding counts of zero, are top-coded as “10+”. <br>


#Outcome Data
**MemberID:**Member pseudonym.<br>
**DaysInHospital_Y2:**Days in hospital, the main outcome, for members with claims in Y1. Values  above 14 days (the 99% percentile) are top-coded as “15+”. <br>
**DaysInHospital_Y3:** Days in hospital, the main outcome, for members with claims in Y2. Values  above 14 days (the 99% percentile) are top-coded as “15+”. <br>
**ClaimedTruncated:** Members with truncated claims in the year prior to the main outcome are  assigned a value of 1, and 0 otherwise. <br>





In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,mean_absolute_error,r2_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import re

In [ ]:
claims_df=pd.read_csv('/content/drive/MyDrive/Courses/Advance business analytics/Assignment 2/Health Challenge/Claims.csv')
daysinhospitaly2_df=pd.read_csv('/content/drive/MyDrive/Courses/Advance business analytics/Assignment 2/Health Challenge/DaysInHospital_Y2.csv')
daysinhospitaly3_df=pd.read_csv('/content/drive/MyDrive/Courses/Advance business analytics/Assignment 2/Health Challenge/DaysInHospital_Y3.csv')
drugC_df=pd.read_csv('/content/drive/MyDrive/Courses/Advance business analytics/Assignment 2/Health Challenge/DrugCount.csv')
labC_df=pd.read_csv('/content/drive/MyDrive/Courses/Advance business analytics/Assignment 2/Health Challenge/LabCount.csv')
lookupPriC_df=pd.read_csv('/content/drive/MyDrive/Courses/Advance business analytics/Assignment 2/Health Challenge/Lookup PrimaryConditionGroup.csv')
lookupPro_df=pd.read_csv('/content/drive/MyDrive/Courses/Advance business analytics/Assignment 2/Health Challenge/Lookup ProcedureGroup.csv')
members_df=pd.read_csv('/content/drive/MyDrive/Courses/Advance business analytics/Assignment 2/Health Challenge/Members.csv')
target_df=pd.read_csv('/content/drive/MyDrive/Courses/Advance business analytics/Assignment 2/Health Challenge/Target.csv')
sampleentry_df=pd.read_csv('/content/drive/MyDrive/Courses/Advance business analytics/Assignment 2/Health Challenge/sampleentry.csv')

In [ ]:
print(claims_df.shape)
print(daysinhospitaly2_df.shape)
print(daysinhospitaly3_df.shape)
print(drugC_df.shape)
print(labC_df.shape)
print(lookupPriC_df.shape)
print(lookupPro_df.shape)
print(members_df.shape)
print(target_df.shape)
print(sampleentry_df.shape)

(2668990, 14)
(76038, 3)
(71435, 3)
(818241, 4)
(361484, 4)
(45, 2)
(17, 2)
(113000, 3)
(70942, 3)
(70942, 3)


In [ ]:
claims_df.head()

,MemberID,ProviderID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS
0,42286978,8013252.0,172193.0,37796.0,Y1,Surgery,Office,28,NaN,8- 9 months,NEUMENT,0,MED,0
1,97903248,3316066.0,726296.0,5300.0,Y3,Internal,Office,50,NaN,7- 8 months,NEUMENT,1-2,EM,0
2,2759427,2997752.0,140343.0,91972.0,Y3,Internal,Office,14,NaN,0- 1 month,METAB3,0,EM,0
3,73570559,7053364.0,240043.0,70119.0,Y3,Laboratory,Independent Lab,24,NaN,5- 6 months,METAB3,1-2,SCS,0
4,11837054,7557061.0,496247.0,68968.0,Y2,Surgery,Outpatient Hospital,27,NaN,4- 5 months,FXDISLC,1-2,EM,0


In [ ]:
claims_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2668990 entries, 0 to 2668989
Data columns (total 14 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   MemberID               int64  
 1   ProviderID             float64
 2   Vendor                 float64
 3   PCP                    float64
 4   Year                   object 
 5   Specialty              object 
 6   PlaceSvc               object 
 7   PayDelay               object 
 8   LengthOfStay           object 
 9   DSFS                   object 
 10  PrimaryConditionGroup  object 
 11  CharlsonIndex          object 
 12  ProcedureGroup         object 
 13  SupLOS                 int64  
dtypes: float64(3), int64(2), object(9)
memory usage: 285.1+ MB


In [ ]:
claims_df.isnull().sum()

,0
MemberID,0
ProviderID,16264
Vendor,24856
PCP,7492
Year,0
Specialty,8405
PlaceSvc,7632
PayDelay,0
LengthOfStay,2597392
DSFS,52770


In [ ]:
#Removing Columns such as ProviderID, Vendor, PCPP
claims_df.drop(['ProviderID','Vendor','PCP'],axis=1,inplace=True)

In [ ]:
def safe_mode(x):
    mode_values = x.mode()
    return mode_values.iloc[0] if not mode_values.empty else None

fill_spec=claims_df.groupby(['MemberID',])['Specialty'].transform(safe_mode)
fill_PlaceSvc=claims_df.groupby(['MemberID',])['PlaceSvc'].transform(safe_mode)
fill_DSFS=claims_df.groupby(['MemberID',])['DSFS'].transform(safe_mode)
fill_primaryConditionGroup=claims_df.groupby(['MemberID',])['PrimaryConditionGroup'].transform(safe_mode)
fill_ProcedureGroup=claims_df.groupby(['MemberID',])['ProcedureGroup'].transform(safe_mode)
claims_df['Specialty'].fillna(fill_spec,inplace=True)
claims_df['PlaceSvc'].fillna(fill_PlaceSvc,inplace=True)
claims_df['DSFS'].fillna(fill_DSFS,inplace=True)
claims_df['PrimaryConditionGroup'].fillna(fill_primaryConditionGroup,inplace=True)
claims_df['ProcedureGroup'].fillna(fill_ProcedureGroup,inplace=True)

<ipython-input-8-0deba931b655>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  claims_df['Specialty'].fillna(fill_spec,inplace=True)
<ipython-input-8-0deba931b655>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [ ]:
#lenghofStay column is dropped as 97% of the data is missing in that column
claims_df.drop(['LengthOfStay'],axis=1,inplace=True)

In [ ]:
daysinhospitaly2_df.head()

,MemberID,ClaimsTruncated,DaysInHospital
0,24027423,0,0
1,98324177,0,0
2,33899367,1,1
3,5481382,0,1
4,69908334,0,0


In [ ]:
daysinhospitaly2_df.shape

(76038, 3)

In [ ]:
daysinhospitaly2_df.isnull().sum()

,0
MemberID,0
ClaimsTruncated,0
DaysInHospital,0


In [ ]:
daysinhospitaly3_df.head()

,MemberID,ClaimsTruncated,DaysInHospital
0,90963501,0,0
1,85160905,0,0
2,45496648,0,0
3,45162557,0,0
4,68793937,0,0


In [ ]:
daysinhospitaly3_df.isnull().sum()

,0
MemberID,0
ClaimsTruncated,0
DaysInHospital,0


In [ ]:
drugC_df.head()

,MemberID,Year,DSFS,DrugCount
0,48925661,Y2,9-10 months,7+
1,90764620,Y3,8- 9 months,3
2,61221204,Y1,2- 3 months,1
3,63628544,Y3,1- 2 months,1
4,46949606,Y2,10-11 months,3


In [ ]:
drugC_df.isnull().sum()

,0
MemberID,0
Year,0
DSFS,0
DrugCount,0


In [ ]:
labC_df.head()

,MemberID,Year,DSFS,LabCount
0,69258001,Y3,2- 3 months,1
1,10143167,Y1,0- 1 month,2
2,1054357,Y1,0- 1 month,6
3,56583841,Y3,6- 7 months,4
4,70967047,Y2,0- 1 month,2


In [ ]:
labC_df.isnull().sum()

,0
MemberID,0
Year,0
DSFS,0
LabCount,0


In [ ]:
lookupPriC_df.head()

,PrimaryConditionGroup,Description
0,AMI,Acute myocardial infarction
1,APPCHOL,Appendicitis
2,ARTHSPIN,Arthropathies
3,CANCRA,Cancer A
4,CANCRB,Cancer B


In [ ]:
lookupPriC_df.isnull().sum()

,0
PrimaryConditionGroup,0
Description,0


In [ ]:
lookupPro_df.head()

,ProcedureGroup,Description
0,ANES,Anesthesia
1,EM,Evaluation and Management
2,MED,Medicine
3,PL,Pathology and Laboratory
4,RAD,Radiology


In [ ]:
lookupPro_df.isnull().sum()

,0
ProcedureGroup,0
Description,0


In [ ]:
members_df.head()

,MemberID,AgeAtFirstClaim,Sex
0,14723353,70-79,M
1,75706636,70-79,M
2,17320609,70-79,M
3,69690888,40-49,M
4,33004608,0-9,M


In [ ]:
members_df.isnull().sum()

,0
MemberID,0
AgeAtFirstClaim,5753
Sex,17552


In [ ]:
target_df.head()

,MemberID,ClaimsTruncated,DaysInHospital
0,20820036,0,NaN
1,14625274,1,NaN
2,99227820,0,NaN
3,74486714,0,NaN
4,92341995,0,NaN


In [ ]:
target_df.isnull().sum()

,0
MemberID,0
ClaimsTruncated,0
DaysInHospital,70942


In [ ]:
sampleentry_df.head()

,MemberID,ClaimsTruncated,DaysInHospital
0,20820036,0,0.203323
1,14625274,1,0.316155
2,99227820,0,3.129290
3,74486714,0,0.124776
4,92341995,0,0.145837


In [ ]:
sampleentry_df.isnull().sum()

,0
MemberID,0
ClaimsTruncated,0
DaysInHospital,0


In [ ]:
merged_df=pd.merge(claims_df,members_df,on='MemberID',how='inner')

In [ ]:
merged_df.shape

(2668990, 12)

In [ ]:
merged_df.isnull().sum()

,0
MemberID,0
Year,0
Specialty,27
PlaceSvc,35
PayDelay,0
DSFS,33
PrimaryConditionGroup,30
CharlsonIndex,0
ProcedureGroup,3
SupLOS,0


In [ ]:
merged_df.head()

,MemberID,Year,Specialty,PlaceSvc,PayDelay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS,AgeAtFirstClaim,Sex
0,42286978,Y1,Surgery,Office,28,8- 9 months,NEUMENT,0,MED,0,80+,F
1,97903248,Y3,Internal,Office,50,7- 8 months,NEUMENT,1-2,EM,0,70-79,NaN
2,2759427,Y3,Internal,Office,14,0- 1 month,METAB3,0,EM,0,40-49,M
3,73570559,Y3,Laboratory,Independent Lab,24,5- 6 months,METAB3,1-2,SCS,0,50-59,F
4,11837054,Y2,Surgery,Outpatient Hospital,27,4- 5 months,FXDISLC,1-2,EM,0,80+,F


In [ ]:
merged_df=pd.merge(merged_df,drugC_df,on=['MemberID','Year','DSFS'],how='left')

In [ ]:
merged_df.shape

(2668990, 13)

In [ ]:
merged_df.head()

,MemberID,Year,Specialty,PlaceSvc,PayDelay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS,AgeAtFirstClaim,Sex,DrugCount
0,42286978,Y1,Surgery,Office,28,8- 9 months,NEUMENT,0,MED,0,80+,F,3
1,97903248,Y3,Internal,Office,50,7- 8 months,NEUMENT,1-2,EM,0,70-79,NaN,7+
2,2759427,Y3,Internal,Office,14,0- 1 month,METAB3,0,EM,0,40-49,M,4
3,73570559,Y3,Laboratory,Independent Lab,24,5- 6 months,METAB3,1-2,SCS,0,50-59,F,NaN
4,11837054,Y2,Surgery,Outpatient Hospital,27,4- 5 months,FXDISLC,1-2,EM,0,80+,F,2


In [ ]:
merged_df=pd.merge(merged_df,labC_df,on=['MemberID','Year','DSFS'],how='left')

In [ ]:
merged_df.isnull().sum()

,0
MemberID,0
Year,0
Specialty,27
PlaceSvc,35
PayDelay,0
DSFS,33
PrimaryConditionGroup,30
CharlsonIndex,0
ProcedureGroup,3
SupLOS,0


In [ ]:
merged_df.shape

(2668990, 14)

In [ ]:
fill_DC=merged_df.groupby(['MemberID',])['DrugCount'].transform(safe_mode)
fill_LabC=merged_df.groupby(['MemberID',])['LabCount'].transform(safe_mode)
merged_df['LabCount'].fillna(fill_LabC,inplace=True)
merged_df['DrugCount'].fillna(fill_DC,inplace=True)

<ipython-input-39-5e5a03c453be>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['LabCount'].fillna(fill_LabC,inplace=True)
<ipython-input-39-5e5a03c453be>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

In [ ]:
merged_df.isnull().sum()

,0
MemberID,0
Year,0
Specialty,27
PlaceSvc,35
PayDelay,0
DSFS,33
PrimaryConditionGroup,30
CharlsonIndex,0
ProcedureGroup,3
SupLOS,0


In [ ]:
merged_df['LabCount'].fillna(0,inplace=True)
merged_df['DrugCount'].fillna(0,inplace=True)

In [ ]:
merged_df['Target'] = merged_df['MemberID'].map(
    daysinhospitaly2_df.set_index('MemberID')['DaysInHospital'].gt(0)
).fillna(False).astype(int)
merged_df=pd.merge(merged_df,daysinhospitaly2_df,on=['MemberID'],how='left')

<ipython-input-42-55d314a0e919>:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(False).astype(int)


In [ ]:
print(merged_df.shape)
merged_df['DaysInHospital'].fillna(0,inplace=True)
merged_df.drop(['ClaimsTruncated'],axis=1,inplace=True)

(2668990, 17)


<ipython-input-43-ec3a8f5852f5>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['DaysInHospital'].fillna(0,inplace=True)


In [ ]:
merged_df.isnull().sum()

,0
MemberID,0
Year,0
Specialty,27
PlaceSvc,35
PayDelay,0
DSFS,33
PrimaryConditionGroup,30
CharlsonIndex,0
ProcedureGroup,3
SupLOS,0


In [ ]:
merged_df.head()

,MemberID,Year,Specialty,PlaceSvc,PayDelay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS,AgeAtFirstClaim,Sex,DrugCount,LabCount,Target,DaysInHospital
0,42286978,Y1,Surgery,Office,28,8- 9 months,NEUMENT,0,MED,0,80+,F,3,8,1,2.0
1,97903248,Y3,Internal,Office,50,7- 8 months,NEUMENT,1-2,EM,0,70-79,NaN,7+,2,1,3.0
2,2759427,Y3,Internal,Office,14,0- 1 month,METAB3,0,EM,0,40-49,M,4,4,0,0.0
3,73570559,Y3,Laboratory,Independent Lab,24,5- 6 months,METAB3,1-2,SCS,0,50-59,F,0,4,0,0.0
4,11837054,Y2,Surgery,Outpatient Hospital,27,4- 5 months,FXDISLC,1-2,EM,0,80+,F,2,4,0,0.0


In [ ]:
merged_df['Target'].value_counts()

,count
Target,
0,2133432
1,535558


In [ ]:
filtered_df=merged_df[merged_df['Year']=='Y1']

In [ ]:
merged_df.shape

(2668990, 16)

In [ ]:
filtered_df.shape

(865689, 16)

In [ ]:
filtered_df['Target'].value_counts()

,count
Target,
0,678421
1,187268


In [ ]:
filtered_df.shape

(865689, 16)

In [ ]:
filtered_df.isnull().sum()

,0
MemberID,0
Year,0
Specialty,10
PlaceSvc,15
PayDelay,0
DSFS,12
PrimaryConditionGroup,20
CharlsonIndex,0
ProcedureGroup,0
SupLOS,0


In [ ]:
filtered_df.drop(['MemberID','Year'],axis=1,inplace=True)

<ipython-input-53-add1babbdfe3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(['MemberID','Year'],axis=1,inplace=True)


In [ ]:
filtered_df.isnull().sum()

,0
Specialty,10
PlaceSvc,15
PayDelay,0
DSFS,12
PrimaryConditionGroup,20
CharlsonIndex,0
ProcedureGroup,0
SupLOS,0
AgeAtFirstClaim,78862
Sex,240150


In [ ]:
filtered_df = filtered_df.dropna()

In [ ]:
filtered_df.shape

(593295, 14)

In [ ]:
filtered_df.isnull().sum()

,0
Specialty,0
PlaceSvc,0
PayDelay,0
DSFS,0
PrimaryConditionGroup,0
CharlsonIndex,0
ProcedureGroup,0
SupLOS,0
AgeAtFirstClaim,0
Sex,0


In [ ]:
def handle_range(value):
    if '-' in value:
        # Split the value on the hyphen, convert to integers, and calculate the mean
        numbers = value.split('-')
        return (int(numbers[0]) + int(numbers[1])) / 2
    else:
        # If no hyphen, return the value as is
        return value

In [ ]:
logistic_df=filtered_df.copy()
logistic_df.head()

,Specialty,PlaceSvc,PayDelay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS,AgeAtFirstClaim,Sex,DrugCount,LabCount,Target,DaysInHospital
0,Surgery,Office,28,8- 9 months,NEUMENT,0,MED,0,80+,F,3,8,1,2.0
6,Rehabilitation,Office,162+,0- 1 month,TRAUMA,0,MED,0,40-49,F,4,2,1,1.0
12,Emergency,Urgent Care,22,6- 7 months,HEART2,0,SDS,0,70-79,M,1,6,0,0.0
15,Internal,Office,162+,0- 1 month,MISCHRT,0,EM,0,40-49,M,1,3,0,0.0
18,General Practice,Urgent Care,46,4- 5 months,GIOBSENT,1-2,SCS,0,50-59,F,5,1,0,0.0


In [ ]:
for c in logistic_df.columns:
  print(f"{c}: {logistic_df[c].unique()}")

Specialty: ['Surgery' 'Rehabilitation' 'Emergency' 'Internal' 'General Practice'
 'Laboratory' 'Diagnostic Imaging' 'Pediatrics'
 'Obstetrics and Gynecology' 'Other' 'Anesthesiology' 'Pathology']
PlaceSvc: ['Office' 'Urgent Care' 'Independent Lab' 'Outpatient Hospital'
 'Inpatient Hospital' 'Ambulance' 'Other' 'Home']
PayDelay: ['28' '162+' '22' '46' '24' '17' '35' '49' '16' '29' '74' '135' '23' '36'
 '148' '63' '88' '51' '31' '47' '21' '50' '71' '13' '115' '70' '128' '53'
 '78' '3' '41' '120' '79' '65' '43' '42' '26' '94' '44' '27' '58' '34'
 '85' '48' '62' '122' '127' '38' '25' '45' '60' '12' '52' '14' '39' '55'
 '56' '92' '67' '20' '87' '11' '15' '116' '123' '7' '109' '84' '32' '69'
 '64' '156' '91' '37' '142' '9' '57' '76' '134' '83' '72' '130' '126' '82'
 '114' '102' '136' '97' '10' '33' '66' '30' '77' '101' '155' '157' '80'
 '73' '153' '143' '132' '140' '158' '59' '107' '150' '40' '141' '154'
 '152' '93' '86' '19' '145' '139' '61' '104' '0' '75' '18' '113' '147'
 '121' '96' '54' 

In [ ]:
for c in ['Specialty','PlaceSvc','PrimaryConditionGroup','ProcedureGroup','Sex']:
  dummy=pd.get_dummies(logistic_df[c],dtype=bool).astype(int)
  logistic_df=pd.concat([logistic_df,dummy],axis=1)
  logistic_df.drop([c],axis=1,inplace=True)

In [ ]:
def convert_to_expression(text):
    # Extract numbers from the string using regular expressions
    numbers = re.findall(r'\d+', text)

    # Check if there are exactly two numbers in the string
    if len(numbers) == 2:
        num1, num2 = int(numbers[0]), int(numbers[1])

        # Create the desired expression: num1 + num2 / 2
        result = num1 + num2 / 2
        return result
    else:
        return "Invalid input"

# Apply the function to the 'Duration' column
logistic_df['DSFS'] = logistic_df['DSFS'].apply(convert_to_expression)

In [ ]:
logistic_df['CharlsonIndex']=logistic_df['CharlsonIndex'].apply(handle_range)
logistic_df['AgeAtFirstClaim']=logistic_df['AgeAtFirstClaim'].apply(handle_range)

In [ ]:
logistic_df["162+"]=logistic_df['PayDelay'].apply(lambda x: 1 if x == '162+' else 0)
logistic_df['PayDelay']=logistic_df['PayDelay'].replace('162+',0)
logistic_df["5+"]=logistic_df['CharlsonIndex'].apply(lambda x: 1 if x == '5+' else 0)
logistic_df['CharlsonIndex']=logistic_df['CharlsonIndex'].replace('5+',0)
logistic_df["80+"]=logistic_df['AgeAtFirstClaim'].apply(lambda x: 1 if x == '80+' else 0)
logistic_df['AgeAtFirstClaim']=logistic_df['AgeAtFirstClaim'].replace('80+',0)
logistic_df['7+']=logistic_df['DrugCount'].apply(lambda x: 1 if x == '7+' else 0)
logistic_df['DrugCount']=logistic_df['DrugCount'].replace('7+',0)
logistic_df['10+']=logistic_df['LabCount'].apply(lambda x: 1 if x == '10+' else 0)
logistic_df['LabCount']=logistic_df['LabCount'].replace('10+',0)

<ipython-input-64-08cf8d970d83>:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  logistic_df['AgeAtFirstClaim']=logistic_df['AgeAtFirstClaim'].replace('80+',0)


In [ ]:
logistic_df.head()

,PayDelay,DSFS,CharlsonIndex,SupLOS,AgeAtFirstClaim,DrugCount,LabCount,Target,DaysInHospital,Anesthesiology,...,SO,SRS,SUS,F,M,162+,5+,80+,7+,10+
0,28,12.5,0,0,0.0,3,8,1,2.0,0,...,0,0,0,1,0,0,0,1,0,0
6,0,0.5,0,0,44.5,4,2,1,1.0,0,...,0,0,0,1,0,1,0,0,0,0
12,22,9.5,0,0,74.5,1,6,0,0.0,0,...,0,0,0,0,1,0,0,0,0,0
15,0,0.5,0,0,44.5,1,3,0,0.0,0,...,0,0,0,0,1,1,0,0,0,0
18,46,6.5,1.5,0,54.5,5,1,0,0.0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
logistic_df[['PayDelay','DrugCount','LabCount','CharlsonIndex','DaysInHospital']]=logistic_df[['PayDelay','DrugCount','LabCount','CharlsonIndex','DaysInHospital']].astype(int)

In [ ]:
X=logistic_df.drop(columns=['Target','DaysInHospital'])
y_daysinhospitaly2 = logistic_df['DaysInHospital']
# Split data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y_daysinhospitaly2, test_size=0.2, random_state=44)

model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluate model performance
mae = mean_absolute_error(y_test, y_pred)
rscore=r2_score(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
print(f"R2 Score: {rscore}")

Mean Absolute Error: 0.7794249627968236
R2 Score: 0.31738820939741197


In [ ]:
X = logistic_df.drop(columns=['Target','DaysInHospital'])  # Drop the target column from features
y_admittediny2 = logistic_df['Target']
X.columns = X.columns.astype(str)
# Split data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y_admittediny2, test_size=0.2, random_state=44)

model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8408


In [ ]:
print(f"positives in testing set: {y_test.sum()}")
print(f"Negatives in testing set: {len(y_test) - y_test.sum()}")
conf_matrix = confusion_matrix(y_test, y_pred)
TP = conf_matrix[1, 1]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]
TN = conf_matrix[0, 0]

# Display confusion matrix with TP, FP, FN, TN labels
print(f"Confusion Matrix:")
print(f"TP: {TP}, FP: {FP}")
print(f"FN: {FN}, TN: {TN}")


positives in testing set: 20998
Negatives in testing set: 97661
Confusion Matrix:
TP: 4955, FP: 2848
FN: 16043, TN: 94813
